In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
from train import Trainer, get_args

In [ ]:
model_path = 'trained_models/nMasks_10_nParam_29_canvasInit_black_ckpt.pth'

x = torch.load(model_path)

args = x['args']

args.ngpus = 1
args.logdir = 'runs/test/'

args.ckptPath = model_path

trainer = Trainer(args)

print(f'Loaded model from {model_path}')

In [ ]:
from PIL import Image
from torchvision import transforms as T

im = Image.open('images/1.png').convert('RGB')

val_transform = T.Compose([
    T.Resize(128),
    T.CenterCrop(128),
    T.ToTensor(),
])
ims = val_transform(im).unsqueeze(0)

In [ ]:
trainer.model.eval();
with torch.no_grad():
    output = trainer.model(ims.cuda(), return_=True)

In [ ]:
from IPython.display import display
canvas = torch.zeros(3, 128, 128)

def show(tensor):
    display(T.ToPILImage()(tensor))
    
show(canvas)
show(ims[0])

for i in range(10):
    print('Iteration: ',i)
    mask = output['allMasks'][i][0].cpu()
    color = output['allColors'][i][0].cpu()
    colored_mask = mask * color[:,None,None]
    
    canvas = canvas * (1-mask) + colored_mask
    show(mask)
    show(colored_mask)
    show(canvas)